# [Module 2.1] 파이프라인으로 생성한 엔드포인트 추론

## 필수 사항
- 이 노트북은 sagemaker_inference_container/container-inference/ 의 아래 두개의 노트북을 먼저 실행해야 합니다.
    - 1.1.Build_Docker.ipynb
    - 2.1.Package_Model_Artifact.ipynb
본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.


## 이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 로컬 모드에서 배포 및 추론 테스트
- 3. 세이지 메이커 (호스트 모드)에서 배포 및 추론 테스트
- 4. 세이지 메이커 신규 모델로 앤드포인트 업데이트
- 5. 신규 엔드포인트 생성
- 6. 신규 앤드포인트 생성 후에 Bake Time (예: 5분) 지난 후에 기존 엔드포인트 삭제
- 7. 마지막 신규 엔드포인트 리소스 제거


---    
    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [12]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.
- inference_docker_image: 사용자 정의 도커 이미지
- byom_artifact: 추론 코드를 포함한 모델 아티펙트

In [13]:
%store -r depolyment_endpoint_name

In [21]:
print("depolyment_endpoint_name: \n ", depolyment_endpoint_name)

depolyment_endpoint_name: 
  lambda-deploy-endpoint-11-16-13-57-36


In [14]:
import sagemaker
import boto3

role = sagemaker.get_execution_role()
runtime_client = boto3.Session().client('sagemaker-runtime')



# 2. 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [15]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

##  추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [16]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of batch_size:  256


## 추론할 Paylaod 하나를 생성

In [17]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


payload:  {'user': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'item': [25, 1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]}


# 3. 엔드 포인트 추론
- Boto3 invoke_endpoint() 로 추론

### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [18]:
import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, depolyment_endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

--- 1.6504881381988525 seconds ---
result:  ['[128, 273, 25, 174, 58, 464, 617, 175, 864, 881]']


# 4. 마지막 신규 엔드포인트 리소스 제거
- 첫번째는 실제 리소스를 제제거하지 않고, 관련 오브젝트만을 보여 줍니다.
- 두번째는 신규 생성한 세이지메이커 모델, 엔드포인트 컨피그는 지우지 않고, 앤프포인트만 삭제 합니다.
    - 이유는 업데이트된 앤드포인트에서 사용하기 때문입니다.

In [25]:
import boto3

sm_client = boto3.client('sagemaker')
from inference_utils import delete_endpoint
delete_endpoint(client = sm_client, 
                endpoint_name = depolyment_endpoint_name
               )



ClientError: An error occurred (ValidationException) when calling the DescribeEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-east-1:057716757052:endpoint-config/lambda-deploy-endpoint-11-16-13-57-36".